## Simulations Mai 2025
Quarterly *.pop files used.  
Simulations with lower diameter threshold = 1 cm  
Threshold factor (*.opt file) = 2.1  
Ohne Plugin!

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import getdata
import sortdata
import plotting 
from tabulate import tabulate
from main_frag_and_rest import main_magnitude_cut
from getdata import PopulationType, data_for_one_year_one_seed

base_dir = os.path.join("output", "results_simulation_quarterly_10cm")
os.makedirs(base_dir, exist_ok=True)
getdata.clear_directory(base_dir)

# year ranges, the data of one range ends up in one plot
year_ranges = [np.array([year]) for year in range(2019, 2024)]
seeds = [1]

for years in year_ranges: 
    print(years)
    crs_all_seeds = []
    det_all_seeds = []

    monthly_files_by_year_and_seed = {}

    for seed in seeds: 
        crs_data_single_years = []
        det_data_single_years = []

        for year in years:
            for seed in seeds:
                orbit_files = []
                for orbit in ["geo", "gto", "fol"]:
                    crs_files, det_files = getdata.find_monthly_files("input", year, orbit, seed)
                    orbit_files.append((crs_files, det_files))
                flattened = [f for pair in orbit_files for f in pair]
                monthly_files_by_year_and_seed[(year, seed)] = flattened

            crs_per_orbit = []
            det_per_orbit = []

            for orbit in ["geo", "gto", "fol"]:
                crs_files, det_files = getdata.find_monthly_files("input", year, orbit, seed)

                if len(crs_files) == 0 or len(det_files) == 0:
                    print(f"Warning: No files found for year {year}, orbit {orbit}, seed {seed}")
                    crs_per_orbit.append(np.empty((0,)))
                    det_per_orbit.append(np.empty((0,)))
                    continue

                crs_arrays = [getdata.array_extender(f) for f in crs_files]
                det_arrays = [getdata.array_extender(f) for f in det_files]

                for file, data in zip(det_files, det_arrays):
                    data = np.array(data)
                    print(f"{file}: {data.shape[1] if data.ndim > 1 else 0} objects")

                data_crs = np.hstack(crs_arrays) if crs_arrays else np.empty((0,))
                data_det = np.hstack(det_arrays) if det_arrays else np.empty((0,))

                crs_per_orbit.append(data_crs)
                det_per_orbit.append(data_det)

            crs_data_single_years.append(tuple(crs_per_orbit))
            det_data_single_years.append(tuple(det_per_orbit))

        crs_GEO = np.hstack([triple[0] for triple in crs_data_single_years])
        crs_GTO = np.hstack([triple[1] for triple in crs_data_single_years])
        crs_fol = np.hstack([triple[2] for triple in crs_data_single_years])

        det_GEO = np.hstack([triple[0] for triple in det_data_single_years])
        det_GTO = np.hstack([triple[1] for triple in det_data_single_years])
        det_fol = np.hstack([triple[2] for triple in det_data_single_years])

        crs_all_seeds.append((crs_GEO, crs_GTO, crs_fol))
        det_all_seeds.append((det_GEO, det_GTO, det_fol))

    year_label = years[0]
    years_dir = os.path.join(base_dir, f"results_{year_label}")
    os.makedirs(years_dir, exist_ok=True)
    getdata.clear_directory(years_dir)

    all_GEO_det = np.hstack([triple[0] for triple in det_all_seeds])
    all_GTO_det = np.hstack([triple[1] for triple in det_all_seeds])
    all_fol_det = np.hstack([triple[2] for triple in det_all_seeds])

    all_GEO_crs = np.hstack([triple[0] for triple in crs_all_seeds])
    all_GTO_crs = np.hstack([triple[1] for triple in crs_all_seeds])
    all_fol_crs = np.hstack([triple[2] for triple in crs_all_seeds])

    num_GEO, num_GTO, num_fol = getdata.data_monthly_one_seed(
        [all_GEO_crs, all_GTO_crs, all_fol_crs],
        [all_GEO_det, all_GTO_det, all_fol_det],
        years,
        years_dir,
        "",
        seeds,
        monthly_files_by_year_and_seed
    )

    combined_data_det = np.hstack([arr for arr in [all_GEO_det, all_GTO_det, all_fol_det] if arr.size > 0])
    print("Combined detected shape:", combined_data_det.shape)

    TLE, frag, rest = sortdata.data_sorter(combined_data_det, 8, 10, 9, 20, 3, 1)

    print("TLE shape:", TLE.shape)
    print("Fragment shape:", frag.shape)
    print("Rest shape:", rest.shape)

    if frag.size > 0 and rest.size > 0:
        combined_data_det = np.vstack([frag, rest])
    elif frag.size > 0:
        combined_data_det = frag
    elif rest.size > 0:
        combined_data_det = rest
    else:
        print("Both arrays are empty. No data to combine.")
        combined_data_det = np.array([])

    print("Filtered combined shape:", combined_data_det.shape)

[2019]
input\stat_Master_19_geo_s1_02.det: 597 objects
input\stat_Master_19_geo_s1_05.det: 563 objects
input\stat_Master_19_geo_s1_11.det: 713 objects
input\stat_Master_19_gto_s1_02.det: 692 objects
input\stat_Master_19_gto_s1_05.det: 684 objects
input\stat_Master_19_gto_s1_11.det: 534 objects
input\stat_Master_19_fol_s1_02.det: 257 objects
input\stat_Master_19_fol_s1_05.det: 299 objects
input\stat_Master_19_fol_s1_11.det: 274 objects
Direct filenames ['input\\stat_Master_19_geo_s1_02.crs', 'input\\stat_Master_19_gto_s1_02.crs', 'input\\stat_Master_19_fol_s1_02.crs', 'input\\stat_Master_19_geo_s1_02.det', 'input\\stat_Master_19_gto_s1_02.det', 'input\\stat_Master_19_fol_s1_02.det']
Direct filenames ['input\\stat_Master_19_geo_s1_05.crs', 'input\\stat_Master_19_gto_s1_05.crs', 'input\\stat_Master_19_fol_s1_05.crs', 'input\\stat_Master_19_geo_s1_05.det', 'input\\stat_Master_19_gto_s1_05.det', 'input\\stat_Master_19_fol_s1_05.det']
Direct filenames ['input\\stat_Master_19_geo_s1_11.crs', 

<Figure size 640x480 with 0 Axes>